Let's start from importing the dummy **dataset** with the columns assumptions:

In [145]:
import pandas as pd
from matplotlib import pyplot as plt

df = pd.read_csv('Dataset_test.csv', low_memory = False)

Then let's create a dataframe containing only line items associated with completed orders. And secondly group the data so that **every row corresponds to an order** placed by a customer.

In [118]:
line_orders = df[~df.Order_id.isnull()]
orders = df.groupby(['Order_id', 'Merchant', 'Datetime', 'Country', 'Shipping Zip']).agg({
    'Blink_shipping': 'sum',
    'Cap_eligibility': 'sum',
    'Inventory_av_eligibility': 'sum',
    'New_to_blink': 'sum',
    'New_to_merch': 'sum',
    'New_to_blink': 'sum',
    'Lineitem_quantity': 'sum',
    'Lineitem_price': 'sum',
    'Shipping_price': 'max'
}).reset_index()

orders['date'] = pd.to_datetime(orders.Datetime).dt.date
orders['day_of_week'] = pd.to_datetime(orders.Datetime).dt.day_name()
orders['hour_of_day'] = pd.to_datetime(orders.Datetime).dt.hour

print(orders.head())

  Order_id  Merchant                   Datetime Country Shipping Zip  \
0   #11946  Ketofood  2021-11-15 13:13:59 +0100      IT        20851   
1   #11947  Ketofood  2021-11-15 14:15:11 +0100      IT        90136   
2   #11948  Ketofood  2021-11-15 16:07:40 +0100      IT        34076   
3   #11949  Ketofood  2021-11-15 16:43:09 +0100      IT        10143   
4   #11950  Ketofood  2021-11-15 18:37:50 +0100      IT        41043   

   Blink_shipping  Cap_eligibility  Inventory_av_eligibility  New_to_blink  \
0             0.0              1.0                       1.0           0.0   
1             0.0              1.0                       0.0           0.0   
2             0.0              1.0                       1.0           1.0   
3             0.0              0.0                       1.0           0.0   
4             0.0              1.0                       0.0           0.0   

   New_to_merch  Lineitem_quantity  Lineitem_price  Shipping_price  \
0           1.0             

In [131]:
#Parameters {for multiple values use a list [item1, item2]}

country_list = ['IT']
merchant_list = ['Ketofood']
date_start = '2021-11-20'
date_end = '2022-11-30'
day_of_week = 'Monday'
hod_start = 12
hod_end = 15

date_start = pd.to_datetime(date_start)
date_end = pd.to_datetime(date_end)

In [130]:
orders = orders[
    orders.Country.isin(country_list) & 
    orders.Merchant.isin(merchant_list) & 
    (orders.date >= date_start) &
    (orders.date < date_end) &
    (orders.hour_of_day >= hod_start) &
    (orders.hour_of_day < hod_end)
    ].reset_index(drop = True)

From here we can compute the **KPIs**:

**1. Percentage of serviceable orders** 
<p> <i>Description</i>: number of orders where Blink is shown at checkout divided by the total number of orders </p>

In [143]:
perc_serv_ord = orders.groupby(['Cap_eligibility', 'Inventory_av_eligibility']).Cap_eligibility.count()

print(kpi1.head())

Cap_eligibility  Inventory_av_eligibility
0.0              0.0                         149
                 1.0                         194
1.0              0.0                         213
                 1.0                         181
Name: Cap_eligibility, dtype: int64
